In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload


import sys, os, inspect
import sys
sys.path.append('/Users/emigardiner/VICO/pjams-ionization/pjams/')

from zeusmp_snapshot_reader import read_zeusmp_snapshot
from zeusmp_snapshot_reader import ScaleFactors
from snapshot import snapshot 
from basic_snapshot import basic_snapshot, VICO_loc
import plot as plot
import slices as slices
from ionfrac import VMIN, VMAX
import flux as flux
import ionfrac as ionfrac

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm
from matplotlib.colors import TwoSlopeNorm

from PIL import Image, ImageDraw, ImageFont
from IPython.display import Image as DisplayImage

In [ ]:
saveloc = VICO_loc + '/pillowfiles/res_maps_fig'

In [ ]:
freqs = np.array([.01, .05, .1, .5, 1, 5.3, 23, 43, 100, 230 ]) # GHz
freqs *= 10**9 # Hz

r_kpc = 1
heights_and_scales = np.load(VICO_loc+'/Data/heights_and_scales.npz')
scales = heights_and_scales['scales'] # AU
heights = heights_and_scales['heights'] # AU  

colors = plot.COLORS

In [ ]:
rnums = np.array([9, 21, 39, 54])
ryears = np.array(['9,000 yrs', '21,000 yrs', '39,000 yrs', '54,000 yrs',])
rmasss = np.array([r'2 M$_\odot$', r'4 M$_\odot$', r'8 M$_\odot$', r'12 M$_\odot$',])
snaps_lr, snaps_mr, snaps_hr = flux.prep_res_snapshots()

mass_lr = np.array(['2 M$_\odot$ lr', '4 M$_\odot$ lr', '8 M$_\odot$ lr', '12 M$_\odot$ lr',])
mass_mr = np.array(['2 M$_\odot$ mr', '4 M$_\odot$ mr', '8 M$_\odot$ mr', '12 M$_\odot$ mr',])
mass_hr = np.array(['2 M$_\odot$ hr', '4 M$_\odot$ hr', '8 M$_\odot$ hr', '12 M$_\odot$ hr',])
# print(mass_lr, mass_mr, mass_hr)

In [ ]:
allsnaps = np.array([snaps_hr, snaps_mr, snaps_lr])
allsnaps = allsnaps.swapaxes(0,1).flatten()
for snap in allsnaps:
    print(snap.name)

In [ ]:
allmasss = np.array([r'2$M_\odot$'+'\nhi', r'2$M_\odot$'+'\nmed', r'2$M_\odot$'+'\nlo',
                         r'4$M_\odot$'+'\nhi', r'4$M_\odot$'+'\nmed', r'4$M_\odot$'+'\nlo', 
                         r'8$M_\odot$'+'\nhi', r'8$M_\odot$'+'\nmed', r'8$M_\odot$'+'\nlo',
                         r'12$M_\odot$'+'\nhi', r'12$M_\odot$'+'\nmed', r'12$M_\odot$'+'\nlo'])

allyears = np.array(['9,000 yrs hi-res', '9,000 yrs mid-res', '9,000 yrs lo-res',
                        '21,000 yrs hi-res', '21,000 yrs mid-res', '21,000 yrs lo-res',
                        '39,000 yrs hi-res', '39,000 yrs mid-res', '39,000 yrs lo-res',
                        '54,000 yrs hi-res', '54,000 yrs mid-res', '54,000 yrs lo-res'])

In [ ]:
for snap in allsnaps.flatten():
    ionfrac.load_average_ionfrac_array(snap)
    ionfrac.load_average_ionfrac_array_ratio(snap, const=False)
    
# Maximum Velocities for Contours
for snap in allsnaps.flatten(): 
    for v in ionfrac.v_mins:
        ionfrac.max_velocity_proj_array(snap, v)

In [ ]:
for Snap in FlatResSnapshots:
    load_average_ionfrac_array(Snap)
    load_average_ionfrac_array_const(Snap, const=False)

# Plotting Functions